# Cafe Sales Data Wrangling Project

This project demonstrates data wrangling and preprocessing skills using a publicly available café sales dataset. The focus is cleaning messy raw data, standardizing formats, handling missing values, and preparing the dataset for further analysis.

Note: This project does not include analysis or modeling; it is purely focused on data wrangling.

## About the Dataset

Source: [Kaggle – Cafe Sales Dataset](https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training)

The Cafe Sales dataset contains 10,000 rows of synthetic data representing sales transactions in a cafe. This dataset is intentionally "dirty," with missing values, inconsistent data, and errors.

Let's view the dataset first!

In [392]:
# First import the required packages
import pandas as pd

import warnings
warnings.filterwarnings('ignore') # To hide package warnings


## Loading and Checking the Raw Data

In [393]:
df = pd.read_csv('../data/raw_cafe_sales.csv')

# view the portion of data
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [394]:
# let's see metadata of the dataset
df.info

<bound method DataFrame.info of      Transaction ID      Item Quantity Price Per Unit Total Spent  \
0       TXN_1961373    Coffee        2            2.0         4.0   
1       TXN_4977031      Cake        4            3.0        12.0   
2       TXN_4271903    Cookie        4            1.0       ERROR   
3       TXN_7034554     Salad        2            5.0        10.0   
4       TXN_3160411    Coffee        2            2.0         4.0   
...             ...       ...      ...            ...         ...   
9995    TXN_7672686    Coffee        2            2.0         4.0   
9996    TXN_9659401       NaN        3            NaN         3.0   
9997    TXN_5255387    Coffee        4            2.0         8.0   
9998    TXN_7695629    Cookie        3            NaN         3.0   
9999    TXN_6170729  Sandwich        3            4.0        12.0   

      Payment Method  Location Transaction Date  
0        Credit Card  Takeaway       2023-09-08  
1               Cash  In-store       20

In [395]:

# Print Data attributes
print("Total number of attributes : ", len(df.columns))
print("Attributes of Dataset : ", list(df.columns))
print("Empty values for attributes", df.isnull().sum())
print("Any duplicate record: ", df.duplicated().sum())


Total number of attributes :  8
Attributes of Dataset :  ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method', 'Location', 'Transaction Date']
Empty values for attributes Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64
Any duplicate record:  0


## Exploring each attribute one by one to see what need to be fixed or transformed


In [396]:
# helper functions

import pandas as pd

def check_attribute_quality(df, attribute):
    """
    Performs basic data quality checks on a specified DataFrame column.

    This function prints:
    - Whether the attribute is numeric or not
    - Number of null (missing) values
    - Number of duplicate values

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing the data.
    attribute : str
        The column name on which data quality checks are performed.

    Returns
    -------
    None
        Prints data quality information to the console.
    """
    
    print(f"\n🔍 Attribute: {attribute}")
    print("-" * 40)
    print("Random Sample data from the attribute: ", list(df[attribute].sample(n=100, random_state=42)))
    print("-" * 40)
    print("Is Numeric:",
          pd.api.types.is_numeric_dtype(df[attribute]))
    
    print("Null values:",
          df[attribute].isnull().sum())
    
    print("Duplicate values:",
          df[attribute].duplicated().sum())
    
    print("Are there 'UNKNOWN' values in the Data:",
          'UNKNOWN' in df[attribute].values)
    
    print("Are there 'ERROR' values in the Data:",
          'ERROR' in df[attribute].values)
    

    
def remove_null_values(df, attribute):
    """
    Removes records from a DataFrame where the specified attribute has null values.
    Also convert the invalid values to Nan values.
    
    This function drops rows containing missing (NaN) values in the given column
    and reports how many records were removed as part of the cleaning process.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame from which records with null values will be removed.
    attribute : str
        The column name to check for null values.

    Returns
    -------
    pandas.DataFrame
        A cleaned DataFrame with records containing null values in the specified
        attribute removed.
    """
    #Replace the invalid values with null values
    df["Item"].replace(['ERROR', 'UNKNOWN'], pd.NA, inplace=True)
    
    before = len(df)
    df = df.dropna(subset=[attribute])
    after = len(df)
    print("Total records before removal:", before)
    print(f"Removed {before - after} rows with missing {attribute} values")
    print("Now the total records are:", after)

    return df



def clean_price_per_unit(df, attribute='Price Per Unit'):
    """
    Cleans the Price Per Unit column

    Steps:
    1. Converts the column to numeric (coercing errors to NaN)
    2. Replace the invalid values to NaN and removes rows with null or non-positive values
    3. Recalculates Total Spent based on Quantity * Price Per Unit

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing the sales data.
    attribute : str, default 'Price Per Unit'
        The column name for price values.

    Returns
    -------
    pandas.DataFrame
        A cleaned DataFrame with valid Price Per Unit values and consistent Total Spent.
    """
    
    before = len(df)
    
    # 1. Convert to numeric
    df[attribute] = pd.to_numeric(df[attribute], errors='coerce')

    #Replace the invalid values with null values
    df[attribute].replace(['ERROR', 'UNKNOWN'], pd.NA, inplace=True)
    
    # Drop rows with missing or non-positive prices
    df = df[df[attribute] > 0]
    
    after = len(df)
    print(f"Removed {before - after} rows with invalid or missing {attribute} values")
    print("Total records after cleaning:", after)
    
    # 3. Recalculate Total Spent
    if 'Quantity' in df.columns and 'Total Spent' in df.columns:
        df['Total Spent'] = df['Quantity'] * df[attribute]
    
    return df


def clean_column(df, attribute):
    """
    Cleans the Payment Method/location column based on input 

    Steps:
    Replaces invalid entries ('Unknown', 'Error') with NaN and then put "UNKNOWN" for all missing values
    Standardizes text (capitalization, removes extra spaces)

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing sales data.
    attribute : str, default 'Payment Method'
        The column name to clean.

    Returns
    -------
    pandas.DataFrame
        A cleaned DataFrame with  Payment Method values.
    """
    
    before = len(df)    
    
    # Replace invalid values with NaN
    df[attribute] .replace(['ERROR'], pd.NA, inplace=True)

    # Replace NaN / NULL with 'UNKNOWN'
    df[attribute] = df[attribute].fillna('UNKNOWN')

    # Standardize text
    df[attribute] = df[attribute].astype(str).str.strip().str.title()    
    return df



def clean_transaction_date(df, attribute='Transaction Date', remove_future=True):
    """
    Cleans the Transaction Date column 

    Steps:
    1. Converts the column to datetime, invalid entries become NaT
    2. Removes transactions with future dates

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing sales data.
    attribute : str, default 'Transaction Date'
        The column name to clean.
    remove_future : bool, default True
        Whether to remove rows with dates in the future.

    Returns
    -------
    pandas.DataFrame
        A cleaned DataFrame with valid Transaction Date values.
    """
    
    before = len(df)
    
    # 1. Convert to datetime
    df[attribute] = pd.to_datetime(df[attribute], errors='coerce')
    
    # 2. remove future dates
    if remove_future:
        df = df[df[attribute] <= pd.Timestamp.today()]
    
    after = len(df)
    print(f"Removed {before - after} rows with invalid or future {attribute} values")
    print("Total records after cleaning:", after)
    
    return df



## 1. Transaction Id
<br>Textual data representing the textual transaction Id.

In [397]:
check_attribute_quality(df, 'Transaction ID')



🔍 Attribute: Transaction ID
----------------------------------------
Random Sample data from the attribute:  ['TXN_2919952', 'TXN_4265056', 'TXN_2463115', 'TXN_7619641', 'TXN_9566593', 'TXN_4835929', 'TXN_8320245', 'TXN_3575713', 'TXN_4383002', 'TXN_3656762', 'TXN_1375992', 'TXN_2201857', 'TXN_2710603', 'TXN_4677415', 'TXN_3432433', 'TXN_1721910', 'TXN_3826038', 'TXN_4428252', 'TXN_3202346', 'TXN_5750248', 'TXN_9089045', 'TXN_7051359', 'TXN_5039782', 'TXN_7764175', 'TXN_8798330', 'TXN_4365321', 'TXN_5186460', 'TXN_4431264', 'TXN_6660602', 'TXN_8973007', 'TXN_6795640', 'TXN_9202496', 'TXN_1339158', 'TXN_6755795', 'TXN_8970187', 'TXN_4268167', 'TXN_7313328', 'TXN_7803615', 'TXN_4509329', 'TXN_8811945', 'TXN_6361856', 'TXN_9130559', 'TXN_5530180', 'TXN_4657842', 'TXN_2354774', 'TXN_6688524', 'TXN_7894558', 'TXN_5650608', 'TXN_8092496', 'TXN_3722720', 'TXN_7180164', 'TXN_5315818', 'TXN_2010658', 'TXN_4995281', 'TXN_6751134', 'TXN_1103495', 'TXN_5005592', 'TXN_6219977', 'TXN_1336900', 'TXN


<span style="color:Green;">_There are no Duplicates or null values for Transaction Id and also by looking at the actual data, this does not need any modification._
</span>



## 2. Item
<br> Name of the item being sold in the transaction!

In [398]:
check_attribute_quality(df, 'Item')


🔍 Attribute: Item
----------------------------------------
Random Sample data from the attribute:  ['Juice', 'Juice', 'Sandwich', 'Juice', 'Cake', 'Coffee', 'Tea', nan, 'Coffee', 'Sandwich', 'Coffee', 'Smoothie', 'Smoothie', 'Cake', 'Cookie', 'Juice', 'Salad', 'Sandwich', 'Salad', 'Salad', 'Coffee', 'Coffee', 'Tea', 'Salad', nan, 'Salad', 'Juice', 'Smoothie', nan, 'Coffee', 'Cookie', 'Cake', 'Salad', 'Smoothie', 'Cake', 'Smoothie', 'Juice', 'Salad', 'Tea', 'ERROR', 'Cookie', 'Sandwich', 'Cookie', 'ERROR', 'Coffee', 'Coffee', 'Coffee', 'Juice', 'Juice', 'Salad', 'Sandwich', 'Salad', 'Smoothie', 'Salad', 'Sandwich', 'Juice', 'Coffee', 'Cookie', 'Coffee', 'Coffee', 'Salad', 'Sandwich', 'Smoothie', 'Cookie', 'Cookie', 'Smoothie', 'Sandwich', 'Juice', 'Tea', 'Cookie', 'Tea', 'UNKNOWN', 'Cake', 'Salad', 'Cake', 'Cookie', 'Tea', 'Salad', 'Coffee', 'Coffee', 'Cake', 'Salad', 'Juice', 'Juice', 'Cookie', 'Cake', 'UNKNOWN', 'Salad', 'Cookie', 'Sandwich', 'Juice', 'Sandwich', 'ERROR', 'Sandwich',


<span style="color:Red;">_There are Duplicates and null values for Items , so let's clean the data for items._
</span>
<br>Duplicates are fine because transactions are unique, but we need to handle the blank values and there are values where "UNKNOWN", "ERROR" is written.

**Steps**
* Replace "UNKNOWN" and "ERROR" with NaN.
* Drop the records for Null Values.

In [399]:
# drop the records for null values
df = remove_null_values(df, 'Item')
check_attribute_quality(df, 'Item')

Total records before removal: 10000
Removed 969 rows with missing Item values
Now the total records are: 9031

🔍 Attribute: Item
----------------------------------------
Random Sample data from the attribute:  ['Sandwich', 'Tea', 'Juice', 'Cake', 'Juice', 'Coffee', 'Cookie', 'Salad', 'Cookie', 'Cake', 'Sandwich', 'Cake', 'Smoothie', 'Smoothie', 'Tea', 'Sandwich', 'Coffee', 'Cake', 'Cake', 'Tea', 'Juice', 'Coffee', 'Tea', 'Tea', 'Cookie', 'Tea', 'Cake', 'Tea', 'Coffee', 'Tea', 'Sandwich', 'Tea', 'Salad', 'Sandwich', 'Juice', 'Cake', 'Salad', 'Salad', 'Cake', 'Cookie', 'Cookie', 'Coffee', 'Sandwich', 'Salad', 'Juice', 'Cookie', 'Cookie', 'Smoothie', 'Coffee', 'Smoothie', 'Tea', 'Sandwich', 'Salad', 'Coffee', 'Cookie', 'Smoothie', 'Smoothie', 'Smoothie', 'Smoothie', 'Cookie', 'Coffee', 'Salad', 'Cake', 'Sandwich', 'Smoothie', 'Coffee', 'Sandwich', 'Coffee', 'Smoothie', 'Cookie', 'Sandwich', 'Tea', 'Sandwich', 'Cake', 'Coffee', 'Smoothie', 'Salad', 'Tea', 'Sandwich', 'Smoothie', 'Smoothie'


## 3. Quantity
<br> Represents how many units of the Item are being sold.

In [400]:
check_attribute_quality(df, 'Quantity')


🔍 Attribute: Quantity
----------------------------------------
Random Sample data from the attribute:  ['4', '1', '1', '3', '1', '1', '4', '3', 'ERROR', '1', '5', '2', '5', '3', '3', '5', '5', 'ERROR', '5', '2', '2', '4', '1', '4', '2', '1', '5', '4', '4', '5', '3', '2', '4', '1', '3', '2', '5', '3', '1', '5', '4', '2', '2', '3', '2', '3', '1', 'ERROR', '2', '4', 'UNKNOWN', '1', '5', '1', '2', '1', '4', '4', '1', '5', '5', '1', '4', '3', '1', '3', '5', 'ERROR', 'UNKNOWN', '3', '5', 'UNKNOWN', '3', '3', '5', '1', '5', '3', '1', '2', '1', '3', '1', '1', '2', '2', '2', '2', '3', '3', '5', '5', '5', '5', '5', '3', '5', '4', '1', '3']
----------------------------------------
Is Numeric: False
Null values: 121
Duplicate values: 9023
Are there 'UNKNOWN' values in the Data: True
Are there 'ERROR' values in the Data: True


**As the Quantity attribute is not numeric, we need to convert it to numeric to perform analysis later.**

Duplicates does not matter for Quantity!

We do have null values which is something need to be handled as something must have been sold for the particular transaction!
Replace invalid values to NaN.
Drop records with null values for quantity.

In [401]:
# convert the quantity into numeric data type and convert any errors into NaN
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

In [402]:
# drop the records for null values
df = remove_null_values(df, 'Quantity')
check_attribute_quality(df, 'Quantity')

Total records before removal: 9031
Removed 420 rows with missing Quantity values
Now the total records are: 8611

🔍 Attribute: Quantity
----------------------------------------
Random Sample data from the attribute:  [1.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 5.0, 2.0, 1.0, 1.0, 5.0, 5.0, 2.0, 1.0, 5.0, 2.0, 1.0, 1.0, 4.0, 4.0, 5.0, 4.0, 1.0, 3.0, 5.0, 1.0, 4.0, 3.0, 3.0, 3.0, 5.0, 4.0, 3.0, 2.0, 2.0, 2.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0, 5.0, 4.0, 1.0, 2.0, 2.0, 4.0, 4.0, 3.0, 5.0, 4.0, 4.0, 5.0, 3.0, 2.0, 5.0, 2.0, 2.0, 4.0, 4.0, 5.0, 5.0, 2.0, 3.0, 1.0, 3.0, 2.0, 2.0, 5.0, 1.0, 5.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 3.0, 1.0, 4.0, 2.0, 4.0, 3.0, 5.0, 2.0, 4.0, 3.0, 4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 3.0, 2.0, 4.0, 2.0]
----------------------------------------
Is Numeric: True
Null values: 0
Duplicate values: 8606
Are there 'UNKNOWN' values in the Data: False
Are there 'ERROR' values in the Data: False


## 4. Price Per Unit

In [403]:
check_attribute_quality(df, 'Price Per Unit')


🔍 Attribute: Price Per Unit
----------------------------------------
Random Sample data from the attribute:  ['4.0', '3.0', '1.0', '4.0', '4.0', '1.0', '3.0', '1.0', '1.5', '3.0', '1.5', '1.5', nan, '4.0', '5.0', '4.0', '3.0', '1.0', '4.0', '3.0', '3.0', '2.0', '3.0', '5.0', '5.0', '2.0', '1.0', '3.0', '4.0', '2.0', '1.0', '2.0', '4.0', '1.5', '4.0', '3.0', '4.0', '1.5', '4.0', '5.0', '4.0', nan, '4.0', '4.0', '3.0', '1.0', '3.0', '4.0', '1.5', '1.5', '3.0', '1.5', '3.0', '4.0', '3.0', '5.0', '4.0', '1.5', 'UNKNOWN', '4.0', '4.0', '3.0', '1.5', '1.5', '2.0', '3.0', '2.0', '5.0', '2.0', '1.0', '1.0', '3.0', '4.0', '5.0', '1.5', '3.0', '5.0', '1.5', '4.0', '3.0', '5.0', '5.0', '3.0', '2.0', '1.0', '3.0', '3.0', '3.0', '3.0', '4.0', '4.0', '3.0', '4.0', '3.0', '4.0', nan, '4.0', '1.0', '1.5', '1.5']
----------------------------------------
Is Numeric: False
Null values: 153
Duplicate values: 8602
Are there 'UNKNOWN' values in the Data: True
Are there 'ERROR' values in the Data: True


**As the Price Per Unit attribute is not numeric, we need to convert it to numeric to perform analysis later.**

Duplicates does not matter for Price Per Unit!

We do have null values which is something need to be handled as Item must be worth something for the particular transaction!
Replace invalid values to NaN. and drop rows with null values.


In [404]:
df = clean_price_per_unit(df)
check_attribute_quality(df, 'Price Per Unit')

Removed 464 rows with invalid or missing Price Per Unit values
Total records after cleaning: 8147

🔍 Attribute: Price Per Unit
----------------------------------------
Random Sample data from the attribute:  [3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 1.0, 1.5, 4.0, 2.0, 3.0, 2.0, 5.0, 3.0, 4.0, 4.0, 1.5, 5.0, 5.0, 1.5, 3.0, 1.5, 2.0, 2.0, 3.0, 5.0, 2.0, 2.0, 2.0, 1.5, 4.0, 5.0, 4.0, 3.0, 3.0, 5.0, 3.0, 1.0, 3.0, 1.5, 3.0, 4.0, 3.0, 4.0, 1.0, 3.0, 2.0, 5.0, 1.5, 3.0, 2.0, 3.0, 1.5, 3.0, 1.0, 1.5, 3.0, 3.0, 1.0, 4.0, 5.0, 4.0, 2.0, 3.0, 4.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 1.5, 1.0, 1.0, 4.0, 4.0, 3.0, 2.0, 2.0, 1.5, 5.0, 5.0, 1.5, 4.0, 4.0, 1.5, 3.0, 2.0, 3.0, 3.0, 2.0, 3.0, 4.0, 3.0, 3.0]
----------------------------------------
Is Numeric: True
Null values: 0
Duplicate values: 8141
Are there 'UNKNOWN' values in the Data: False
Are there 'ERROR' values in the Data: False


## 5. Total Spent

The total amount the user spent in the transaction.

In [405]:
check_attribute_quality(df, 'Total Spent')


🔍 Attribute: Total Spent
----------------------------------------
Random Sample data from the attribute:  [6.0, 12.0, 12.0, 8.0, 8.0, 15.0, 4.0, 3.0, 20.0, 6.0, 3.0, 10.0, 15.0, 6.0, 12.0, 20.0, 1.5, 20.0, 5.0, 7.5, 12.0, 4.5, 10.0, 10.0, 12.0, 20.0, 6.0, 8.0, 10.0, 7.5, 20.0, 20.0, 12.0, 6.0, 15.0, 25.0, 12.0, 2.0, 9.0, 6.0, 15.0, 12.0, 3.0, 20.0, 3.0, 6.0, 8.0, 10.0, 3.0, 3.0, 10.0, 9.0, 1.5, 6.0, 3.0, 1.5, 12.0, 6.0, 1.0, 8.0, 15.0, 20.0, 8.0, 9.0, 4.0, 4.0, 9.0, 2.0, 9.0, 15.0, 9.0, 16.0, 6.0, 15.0, 15.0, 8.0, 3.0, 3.0, 1.0, 12.0, 4.0, 3.0, 8.0, 2.0, 3.0, 20.0, 25.0, 4.5, 12.0, 20.0, 3.0, 6.0, 2.0, 6.0, 3.0, 8.0, 15.0, 12.0, 12.0, 15.0]
----------------------------------------
Is Numeric: True
Null values: 0
Duplicate values: 8130
Are there 'UNKNOWN' values in the Data: False
Are there 'ERROR' values in the Data: False


**Data looks good for Total Spent**

## 6. Payment Method

Payment Method user used for the transaction.

In [406]:
check_attribute_quality(df, 'Payment Method')


🔍 Attribute: Payment Method
----------------------------------------
Random Sample data from the attribute:  ['Credit Card', 'Credit Card', 'Digital Wallet', 'Cash', 'Credit Card', 'Credit Card', 'ERROR', nan, 'UNKNOWN', nan, nan, 'Cash', 'Cash', nan, 'Credit Card', 'ERROR', 'UNKNOWN', 'Cash', 'Digital Wallet', nan, 'Credit Card', 'Cash', 'Cash', 'Digital Wallet', 'Cash', nan, 'Digital Wallet', 'Cash', nan, 'Credit Card', nan, 'Credit Card', nan, 'Cash', 'Digital Wallet', 'Credit Card', 'Cash', 'Digital Wallet', 'Credit Card', 'Digital Wallet', nan, 'Cash', 'Cash', 'Digital Wallet', 'Cash', 'ERROR', nan, 'Digital Wallet', nan, 'Digital Wallet', nan, 'Cash', 'Cash', 'Cash', 'Digital Wallet', 'Digital Wallet', 'Digital Wallet', 'Digital Wallet', nan, 'Credit Card', nan, 'UNKNOWN', nan, nan, nan, 'Cash', 'Digital Wallet', 'Digital Wallet', 'Credit Card', 'UNKNOWN', 'Digital Wallet', 'Credit Card', 'Digital Wallet', 'Digital Wallet', 'Credit Card', nan, nan, 'Credit Card', 'Cash', 'Credit

**Fixes**
* As the data is categorical and needs these categories to follow some standards so that it will make analysis easy and efficient.
* Remove extra spaces and Capitilization of the methods.
* Remove Invalid Values and records with "UNKNOWN" because we want to preserve the data even if payment method is not defined. 

In [407]:
df = clean_column(df, attribute='Payment Method')
check_attribute_quality(df, 'Payment Method')


🔍 Attribute: Payment Method
----------------------------------------
Random Sample data from the attribute:  ['Credit Card', 'Credit Card', 'Digital Wallet', 'Cash', 'Credit Card', 'Credit Card', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Cash', 'Cash', 'Unknown', 'Credit Card', 'Unknown', 'Unknown', 'Cash', 'Digital Wallet', 'Unknown', 'Credit Card', 'Cash', 'Cash', 'Digital Wallet', 'Cash', 'Unknown', 'Digital Wallet', 'Cash', 'Unknown', 'Credit Card', 'Unknown', 'Credit Card', 'Unknown', 'Cash', 'Digital Wallet', 'Credit Card', 'Cash', 'Digital Wallet', 'Credit Card', 'Digital Wallet', 'Unknown', 'Cash', 'Cash', 'Digital Wallet', 'Cash', 'Unknown', 'Unknown', 'Digital Wallet', 'Unknown', 'Digital Wallet', 'Unknown', 'Cash', 'Cash', 'Cash', 'Digital Wallet', 'Digital Wallet', 'Digital Wallet', 'Digital Wallet', 'Unknown', 'Credit Card', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Cash', 'Digital Wallet', 'Digital Wallet', 'Credit Card', 'Unknown', 'Digital W

## 7. Location



In [408]:
check_attribute_quality(df, 'Location')


🔍 Attribute: Location
----------------------------------------
Random Sample data from the attribute:  ['In-store', nan, 'In-store', 'In-store', 'UNKNOWN', nan, 'Takeaway', 'Takeaway', nan, 'In-store', 'UNKNOWN', 'In-store', 'Takeaway', nan, nan, nan, nan, nan, 'In-store', nan, nan, 'Takeaway', nan, nan, 'In-store', 'In-store', nan, 'Takeaway', 'In-store', 'UNKNOWN', 'Takeaway', 'Takeaway', 'Takeaway', 'UNKNOWN', 'In-store', 'In-store', 'In-store', nan, nan, 'Takeaway', nan, 'In-store', 'Takeaway', 'In-store', 'In-store', 'In-store', 'In-store', 'UNKNOWN', 'Takeaway', 'Takeaway', 'In-store', nan, nan, 'In-store', nan, 'Takeaway', nan, nan, nan, 'Takeaway', 'Takeaway', nan, 'Takeaway', 'In-store', 'Takeaway', nan, 'Takeaway', nan, 'UNKNOWN', 'Takeaway', nan, nan, 'Takeaway', 'In-store', nan, 'In-store', 'In-store', 'Takeaway', nan, 'In-store', 'Takeaway', 'In-store', 'In-store', nan, 'In-store', 'In-store', 'UNKNOWN', 'Takeaway', nan, 'In-store', nan, 'Takeaway', nan, nan, 'In-store', 

**Fixes**
<br>
1. Replaces invalid entries ('ERROR', 'UNKNOWN') and null values with "UNKNOWN" just to preserve the data
2. Standardizes text (capitalization and removes extra spaces)

In [409]:
df = clean_column(df, 'Location')
check_attribute_quality(df, 'Location')


🔍 Attribute: Location
----------------------------------------
Random Sample data from the attribute:  ['In-Store', 'Unknown', 'In-Store', 'In-Store', 'Unknown', 'Unknown', 'Takeaway', 'Takeaway', 'Unknown', 'In-Store', 'Unknown', 'In-Store', 'Takeaway', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'In-Store', 'Unknown', 'Unknown', 'Takeaway', 'Unknown', 'Unknown', 'In-Store', 'In-Store', 'Unknown', 'Takeaway', 'In-Store', 'Unknown', 'Takeaway', 'Takeaway', 'Takeaway', 'Unknown', 'In-Store', 'In-Store', 'In-Store', 'Unknown', 'Unknown', 'Takeaway', 'Unknown', 'In-Store', 'Takeaway', 'In-Store', 'In-Store', 'In-Store', 'In-Store', 'Unknown', 'Takeaway', 'Takeaway', 'In-Store', 'Unknown', 'Unknown', 'In-Store', 'Unknown', 'Takeaway', 'Unknown', 'Unknown', 'Unknown', 'Takeaway', 'Takeaway', 'Unknown', 'Takeaway', 'In-Store', 'Takeaway', 'Unknown', 'Takeaway', 'Unknown', 'Unknown', 'Takeaway', 'Unknown', 'Unknown', 'Takeaway', 'In-Store', 'Unknown', 'In-Store', 'In-Store', 'Take

## 8. Transaction Date



In [410]:
check_attribute_quality(df, 'Transaction Date')


🔍 Attribute: Transaction Date
----------------------------------------
Random Sample data from the attribute:  ['2023-07-20', '2023-03-23', '2023-04-02', '2023-07-08', '2023-04-17', '2023-10-02', '2023-06-18', 'ERROR', '2023-11-18', '2023-12-24', 'UNKNOWN', '2023-04-18', '2023-06-30', '2023-09-03', '2023-02-25', nan, '2023-06-11', '2023-09-02', '2023-09-03', '2023-08-17', '2023-11-30', '2023-03-17', '2023-09-23', '2023-05-22', '2023-04-06', '2023-09-14', '2023-03-28', 'UNKNOWN', 'UNKNOWN', nan, '2023-08-23', '2023-08-26', '2023-07-11', '2023-05-31', nan, '2023-02-20', '2023-06-16', '2023-05-26', '2023-06-12', '2023-02-13', '2023-05-25', '2023-09-10', '2023-10-13', '2023-08-23', '2023-08-29', '2023-08-05', '2023-09-11', '2023-12-23', '2023-04-01', '2023-11-16', '2023-06-20', '2023-12-08', '2023-02-05', '2023-11-26', nan, '2023-05-24', '2023-09-03', '2023-03-10', '2023-04-28', '2023-09-16', '2023-05-01', '2023-08-16', '2023-12-09', '2023-04-23', '2023-06-30', '2023-12-08', '2023-07-16',

**Fixes**
1. Converts the column to datetime, invalid entries become NaT
2. Removes transactions with future dates

In [411]:
df = clean_transaction_date(df)
check_attribute_quality(df, 'Transaction Date')

Removed 374 rows with invalid or future Transaction Date values
Total records after cleaning: 7773

🔍 Attribute: Transaction Date
----------------------------------------
Random Sample data from the attribute:  [Timestamp('2023-06-01 00:00:00'), Timestamp('2023-10-03 00:00:00'), Timestamp('2023-04-20 00:00:00'), Timestamp('2023-07-24 00:00:00'), Timestamp('2023-07-14 00:00:00'), Timestamp('2023-09-02 00:00:00'), Timestamp('2023-05-16 00:00:00'), Timestamp('2023-04-14 00:00:00'), Timestamp('2023-05-23 00:00:00'), Timestamp('2023-10-31 00:00:00'), Timestamp('2023-05-01 00:00:00'), Timestamp('2023-08-21 00:00:00'), Timestamp('2023-03-05 00:00:00'), Timestamp('2023-12-20 00:00:00'), Timestamp('2023-11-23 00:00:00'), Timestamp('2023-02-26 00:00:00'), Timestamp('2023-02-13 00:00:00'), Timestamp('2023-04-24 00:00:00'), Timestamp('2023-11-24 00:00:00'), Timestamp('2023-04-14 00:00:00'), Timestamp('2023-02-06 00:00:00'), Timestamp('2023-07-15 00:00:00'), Timestamp('2023-10-20 00:00:00'), Timest

# Final Clean Dataset

In [412]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7773 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    7773 non-null   object        
 1   Item              7773 non-null   object        
 2   Quantity          7773 non-null   float64       
 3   Price Per Unit    7773 non-null   float64       
 4   Total Spent       7773 non-null   float64       
 5   Payment Method    7773 non-null   object        
 6   Location          7773 non-null   object        
 7   Transaction Date  7773 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 546.5+ KB


In [413]:
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-Store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-Store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-Store,2023-06-11
...,...,...,...,...,...,...,...,...
9992,TXN_2739140,Smoothie,4.0,4.0,16.0,Unknown,In-Store,2023-07-05
9993,TXN_4766549,Smoothie,2.0,4.0,8.0,Cash,Unknown,2023-10-20
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Unknown,Unknown,2023-08-30
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,Unknown,2023-03-02


In [414]:

# Print Data attributes
print("Total number of attributes : ", len(df.columns))
print("Attributes of Dataset : ", list(df.columns))
print("Empty values for attributes", df.isnull().sum())
print("Any duplicate record: ", df.duplicated().sum())


Total number of attributes :  8
Attributes of Dataset :  ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method', 'Location', 'Transaction Date']
Empty values for attributes Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64
Any duplicate record:  0


# Export Data to clean CSV - cleaned_cafe_sales.csv


In [416]:
# Save Data
df.to_csv('../data/cleaned_cafe_sales.csv', index=False)